In [1]:
'''更新日志09July
这是第一个beta版，已经能无bug运行，排版还没定下来，调参不太方便。
体素模型比预期中快得多，比点云还快。
学习率递减还没写，这可能导致loss在下降中遇到瓶颈。
没有配准的程序，所以现在输入的数据是乱的。
'''

'更新日志09July\n这是第一个beta版，已经能无bug运行，排版还没定下来。\n'

In [2]:
'''
双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。
通道1：原始牙齿的体素模型
通道2：预备体的体素模型
网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出
'''

'\n双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。\n通道1：原始牙齿的体素模型\n通道2：预备体的体素模型\n网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出\n'

In [3]:
'''
# 安装环境
pip install trimesh
'''

'\n# 安装环境\npip install trimesh\n'

In [4]:
'''
# 安装环境
pip install torch torchvision torchaudio
'''

'\n# 安装环境\npip install torch torchvision torchaudio\n'

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import trimesh
from sklearn.model_selection import KFold

/home/jupyter-24143/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 加载并归一化打磨前的初始模型
initial_mesh_path = 'incisor.obj'
initial_mesh = normalize_mesh(load_mesh(initial_mesh_path))
voxel_size = 32  # 体素网格尺寸
voxel_data_initial = mesh_to_voxel(initial_mesh, voxel_size)

class TeethDataset(Dataset):
    def __init__(self, initial_voxel, score_file_path, voxel_size=32):
        self.initial_voxel = initial_voxel
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        # 加载并归一化打磨后的模型
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data_after = mesh_to_voxel(mesh, self.voxel_size)
        # 双通道数据
        sample = np.stack([self.initial_voxel, voxel_data_after], axis=0)
        return torch.tensor(sample, dtype=torch.float32), torch.tensor([score], dtype=torch.float32)  # 注意这里将 score 包装为列表


In [7]:
# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(voxel_data_initial, score_file_path, voxel_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [8]:
# 定义模型
class Teeth3DCNN(nn.Module):
    def __init__(self):
        super(Teeth3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(2, 32, kernel_size=3, stride=1, padding=1)  # 双通道输入
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)  # 体素尺寸为32*32*32，经过两次pooling变为8*8*8
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建模型
model = Teeth3DCNN()
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

In [9]:
'''
10epoch实验中每epochloss不稳定
或许要修改训练方法，遍历每个预备体
'''

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(dataloader)}")


Epoch 1/30, Loss: 783.6944711584794
Epoch 2/30, Loss: 281.69014940763776
Epoch 3/30, Loss: 323.32345892253676
Epoch 4/30, Loss: 269.2934516605578
Epoch 5/30, Loss: 270.2882509733501
Epoch 6/30, Loss: 253.46447462784616
Epoch 7/30, Loss: 304.05945818047775
Epoch 8/30, Loss: 256.2094198528089
Epoch 9/30, Loss: 263.92427795811705
Epoch 10/30, Loss: 245.1622186459993
Epoch 11/30, Loss: 246.5345754623413
Epoch 12/30, Loss: 230.22765149568258
Epoch 13/30, Loss: 257.65727750878585
Epoch 14/30, Loss: 312.2177856344926
Epoch 15/30, Loss: 277.29179231744064
Epoch 16/30, Loss: 197.61830942254318
Epoch 17/30, Loss: 186.5844337312799
Epoch 18/30, Loss: 183.45288060840807
Epoch 19/30, Loss: 173.22488900234825
Epoch 20/30, Loss: 206.01304696735582
Epoch 21/30, Loss: 142.4056820618479
Epoch 22/30, Loss: 136.80933716422632
Epoch 23/30, Loss: 123.6143247202823
Epoch 24/30, Loss: 146.0754273816159
Epoch 25/30, Loss: 128.45888564461157
Epoch 26/30, Loss: 94.71547934883519
Epoch 27/30, Loss: 65.27259342293

In [12]:
'''
# K 折训练

kf = KFold(n_splits=5)
num_epochs = 10
for fold, (train_index, val_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}/{kf.n_splits}")
    
    # 重新初始化模型和优化器
    model = Teeth3DCNN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 创建训练和验证数据子集
    train_subset = torch.utils.data.Subset(dataset, train_index)
    val_subset = torch.utils.data.Subset(dataset, val_index)
    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)
    
    # 训练模型
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # 验证模型
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")
'''

'\n# K 折训练\n\nkf = KFold(n_splits=5)\nnum_epochs = 10\nfor fold, (train_index, val_index) in enumerate(kf.split(dataset)):\n    print(f"Fold {fold + 1}/{kf.n_splits}")\n    \n    # 重新初始化模型和优化器\n    model = Teeth3DCNN()\n    criterion = nn.MSELoss()\n    optimizer = optim.Adam(model.parameters(), lr=0.001)\n    \n    # 创建训练和验证数据子集\n    train_subset = torch.utils.data.Subset(dataset, train_index)\n    val_subset = torch.utils.data.Subset(dataset, val_index)\n    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)\n    val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)\n    \n    # 训练模型\n    for epoch in range(num_epochs):\n        model.train()\n        running_loss = 0.0\n        for inputs, targets in train_loader:\n            optimizer.zero_grad()\n            outputs = model(inputs)\n            loss = criterion(outputs, targets)\n            loss.backward()\n            optimizer.step()\n            running_loss += loss.item()\n        \n        # 验证模型

In [11]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}')


Tooth 1: Score = 73.88922119140625
Tooth 2: Score = 68.90879821777344
Tooth 3: Score = 74.0575180053711
Tooth 4: Score = 79.77029418945312
Tooth 5: Score = 77.80015563964844
Tooth 6: Score = 66.6014404296875
Tooth 7: Score = 75.77994537353516
Tooth 8: Score = 74.26123046875
Tooth 9: Score = 83.14289093017578
Tooth 10: Score = 78.237060546875
Tooth 11: Score = 53.30789566040039
Tooth 12: Score = 66.26764678955078
Tooth 13: Score = 60.1616325378418
Tooth 14: Score = 67.70614624023438
Tooth 15: Score = 56.763851165771484
Tooth 16: Score = 43.71913528442383
Tooth 17: Score = 59.89752960205078
Tooth 18: Score = 63.28911209106445
Tooth 19: Score = 73.5389404296875
Tooth 20: Score = 63.11807632446289
Tooth 21: Score = 72.93743896484375
Tooth 22: Score = 68.06269836425781
Tooth 23: Score = 75.14138793945312
Tooth 24: Score = 70.96456909179688
Tooth 25: Score = 55.82496643066406
Tooth 26: Score = 64.07771301269531
Tooth 27: Score = 67.51405334472656
Tooth 28: Score = 64.04644775390625
Tooth 29:

In [14]:
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )

Tooth 1: Score = 73.88922119140625 Err = -1.11077880859375
Tooth 2: Score = 68.90879821777344 Err = 0.9087982177734375
Tooth 3: Score = 74.0575180053711 Err = 4.057518005371094
Tooth 4: Score = 79.77029418945312 Err = 10.770294189453125
Tooth 5: Score = 77.80015563964844 Err = 1.8001556396484375
Tooth 6: Score = 66.6014404296875 Err = 2.6014404296875
Tooth 7: Score = 75.77994537353516 Err = 6.779945373535156
Tooth 8: Score = 74.26123046875 Err = 1.26123046875
Tooth 9: Score = 83.14289093017578 Err = 6.142890930175781
Tooth 10: Score = 78.237060546875 Err = 1.237060546875
Tooth 11: Score = 53.30789566040039 Err = -0.6921043395996094
Tooth 12: Score = 66.26764678955078 Err = 4.267646789550781
Tooth 13: Score = 60.1616325378418 Err = 1.1616325378417969
Tooth 14: Score = 67.70614624023438 Err = 4.706146240234375
Tooth 15: Score = 56.763851165771484 Err = -0.23614883422851562
Tooth 16: Score = 43.71913528442383 Err = -4.280864715576172
Tooth 17: Score = 59.89752960205078 Err = -0.1024703979